<a href="https://colab.research.google.com/github/toshiki-sakai/rewrite_x_post.ipynb/blob/feature%2Fgemini-pro%E2%86%92flash/rewrite_x_post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================
# ライブラリのインストール
# ========================
# !pip install -q ipywidgets  # ← この行はColabセルで実行してください
import pandas as pd
import requests
import json
import time
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# ========================
# Gemini APIキーの取得方法を表示
# ========================
api_instructions = widgets.HTML(
    value='''
    <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 15px; border-left: 5px solid #4285f4;">
        <h3 style="margin-top: 0; color: #4285f4;">📝 Gemini APIキーの取得方法</h3>
        <ol>
            <li>Google AI Studioにアクセス: <a href="https://aistudio.google.com/app/apikey" target="_blank">https://aistudio.google.com/app/apikey</a></li>
            <li>Googleアカウントでログイン</li>
            <li>「APIキーを作成」をクリック</li>
            <li>キーに名前をつけて（例：「ツイートリライトツール」）、「作成」をクリック</li>
            <li>表示されたAPIキーをコピーして、下のフォームに貼り付け</li>
        </ol>
        <p><strong>注意:</strong> APIキーは秘密情報です。他人と共有しないでください。</p>
    </div>
    ''',
    layout=widgets.Layout(width='100%')
)

# ========================
# ついスポからのCSVダウンロード方法を表示
# ========================
csv_instructions = widgets.HTML(
    value='''
    <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 15px; border-left: 5px solid #34a853;">
        <h3 style="margin-top: 0; color: #34a853;">📊 ついスポからCSVをダウンロードする方法</h3>
        <ol>
            <li>Chrome拡張「ついスポ」をインストール: <a href="https://chromewebstore.google.com/detail/%E3%81%A4%E3%81%84%E3%81%99%E3%81%BD-tweet-export/imjojcgefiinokaclmifakpphkfeiddg?hl=ja" target="_blank">ついスポ - Chrome ウェブストア</a></li>
            <li>Twitterにログイン</li>
            <li>自分のプロフィールページにアクセス</li>
            <li>ついスポアイコンをクリック</li>
            <li>「CSVダウンロード」をクリックしてツイートデータを保存</li>
            <li>下の「実行」ボタンをクリック後、保存したCSVファイルをアップロード</li>
        </ol>
        <p><strong>必要な列:</strong> 「テキスト」「フォロワー数」「imp」の列が含まれていることを確認してください。</p>
    </div>
    ''',
    layout=widgets.Layout(width='100%')
)

# ========================
# APIキー入力フォーム
# ========================
api_key_widget = widgets.Password(
    value='',
    placeholder='ここにGemini APIキーを入力してください',
    description='Gemini API Key:',
    layout=widgets.Layout(width='70%'),
    style={'description_width': 'initial'}
)

# ========================
# カスタムプロンプト入力フォーム
# ========================
custom_prompt_widget = widgets.Textarea(
    value='',
    placeholder='ここにカスタムプロンプトを入力してください。リライトするテキストは[text]で指定してください。',
    description='カスタムプロンプト:',
    layout=widgets.Layout(width='70%', height='100px'),
    style={'description_width': 'initial'}
)

# ========================
# スプレッドシートリンクボタン
# ========================
spreadsheet_link = widgets.HTML(
    value='<a href="https://docs.google.com/spreadsheets/d/1Z6jJ1lGLmyxUqGinY0JAM5SDSFmwDIirOMJzfkw0jO4/edit?usp=sharing" target="_blank">プロンプト一覧スプレッドシート</a>',
    layout=widgets.Layout(width='70%')
)

# ========================
# 実行ボタン
# ========================
run_button = widgets.Button(
    description='実行',
    button_style='success'
)

# 進捗表示用のウィジェット
progress_output = widgets.Output()

# 全ての要素を表示
display(api_instructions, csv_instructions, api_key_widget, custom_prompt_widget, spreadsheet_link, run_button, progress_output)

# ========================
# メイン処理関数
# ========================
def run_rewrite(_):
    with progress_output:
        clear_output()

        API_KEY = api_key_widget.value
        if not API_KEY:
            print("⚠️ APIキーが入力されていません。")
            return

        # 常に20件全てを処理するように固定
        start = 1
        end = 20

        # CSVアップロード
        print("📁 ついスポでダウンロードしたCSVファイルをアップロードしてください")
        uploaded = files.upload()
        if not uploaded:
            print("⚠️ ファイルがアップロードされませんでした。")
            return

        filename = next(iter(uploaded))
        try:
            df = pd.read_csv(filename)
        except Exception as e:
            print(f"⚠️ CSVファイルの読み込みエラー: {e}")
            return

        # 必要なカラムが存在するか確認
        required_columns = ['テキスト', 'フォロワー数', 'imp']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"⚠️ CSVファイルに必要なカラムがありません: {', '.join(missing_columns)}")
            return

        # スコア計算と下位5件抽出
        try:
            df['フォロワー数'] = pd.to_numeric(df['フォロワー数'], errors='coerce')
            df['imp'] = pd.to_numeric(df['imp'], errors='coerce')
            df = df.dropna(subset=['フォロワー数', 'imp'])
            df = df[df['imp'] > 0]
            df['score'] = df['フォロワー数'] / df['imp']

            # スコアが小さい順に並べ替え、最初の5件を取得
            # ascending=True で昇順（小さい順）になります
            target_df = df.sort_values(by='score', ascending=True).head(5).copy().reset_index(drop=True)

            if len(target_df) == 0:
                print("⚠️ 条件に合うデータがありません。")
                return
        except Exception as e:
            print(f"⚠️ データ処理エラー: {e}")
            return

        # 処理対象件数の表示メッセージを修正
        total_items = len(target_df)
        print(f"✅ フォロワー/impが少ない順に上位{total_items}件を処理対象とします。")

        # Geminiリクエスト関数
        def rewrite_text(text):
            url = f"https://generativelanguage.googleapis.com/v1/models/gemini-1.5-flash:generateContent?key={API_KEY}"
            headers = {"Content-Type": "application/json"}
            custom_prompt = custom_prompt_widget.value
            if custom_prompt:
                prompt = custom_prompt.replace('[text]', text)
            else:
                prompt = f"以下の日本語ツイートをより魅力的に140文字以内でリライトしてください：\n{text}"

            # body を明示的に 'contents' のみで定義
            body = {
                "contents": [{"parts": [{"text": prompt}]}]
                # generationConfig が含まれないことを保証
            }

            for attempt in range(3):
                response = requests.post(url, headers=headers, data=json.dumps(body))
                if response.status_code == 200:
                    try:
                        return response.json()['candidates'][0]['content']['parts'][0]['text']
                    except Exception as e:
                        return f"⚠️ 応答形式エラー: {e}"
                elif response.status_code == 429:
                    if attempt < 2:
                        print(f"429エラー: 90秒待機してリトライします（{attempt+1}/3）")
                        time.sleep(90)
                    else:
                        return "❌ エラー: 429（リクエスト過多）"
                else:
                    return f"❌ エラー: {response.status_code}"

        # リライト処理（60秒間隔）
        rewritten = []
        print(f"🔁 選択された{total_items}件をGemini 1.5 Proでリライトします")

        # 処理対象の総数を表示
        total_items = len(target_df)
        print(f"処理対象: {total_items}件")

        for i, row in target_df.iterrows():
            current_position = i + 1
            progress = f"[{current_position}/{total_items}]"

            print(f"\n▶️ {progress} {i+1}件目を処理中... テキスト: {row['テキスト'][:30]}...")

            # リクエスト実行
            result = rewrite_text(row['テキスト'])

            # 結果表示
            if result.startswith("❌") or result.startswith("⚠️"):
                print(f"🔴 {progress} 結果: {result}")
            else:
                print(f"🟢 {progress} 結果: {result[:50]}...")

            rewritten.append(result)

            # 最後のリクエスト以外は待機
            if i < len(target_df) - 1:
                wait_time = 60  # 待機時間を60秒に短縮
                print(f"⏱️ {progress} 次のリクエストまで {wait_time}秒待機します...")
                time.sleep(wait_time)

        # 結果保存
        try:
            target_df['リライト'] = rewritten
            # 必要なカラムを保持し、リライト元の右にリライト結果
            cols = list(target_df.columns)
            if 'リライト' in cols:
                cols.insert(cols.index('テキスト') + 1, cols.pop(cols.index('リライト')))
            target_df = target_df[cols]
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            output_filename = f"rewritten_{timestamp}.csv"
            target_df.to_csv(output_filename, index=False)
            files.download(output_filename)  # 自動でダウンロード
            print("\n✅ 完了しました！CSVをダウンロードしてください。")
        except Exception as e:
            print(f"\n⚠️ 結果保存エラー: {e}")

# 実行ボタンのアクション設定
run_button.on_click(run_rewrite)

HTML(value='\n    <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 15p…

HTML(value='\n    <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 15p…

Password(description='Gemini API Key:', layout=Layout(width='70%'), placeholder='ここにGemini APIキーを入力してください', st…

Textarea(value='', description='カスタムプロンプト:', layout=Layout(height='100px', width='70%'), placeholder='ここにカスタムプ…

HTML(value='<a href="https://docs.google.com/spreadsheets/d/1Z6jJ1lGLmyxUqGinY0JAM5SDSFmwDIirOMJzfkw0jO4/edit?…

Button(button_style='success', description='実行', style=ButtonStyle())

Output()